# Data processing

In [1]:
!pip install imutils

In [4]:
import zipfile
import os
import shutil
import pandas as pd
import ast
import random
from imutils import paths
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
import keras
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle

# if 'tensorflow-great-barrier-reef' not in os.listdir(os.getcwd()):
#     !kaggle competitions download -c tensorflow-great-barrier-reef
#     with zipfile.ZipFile('tensorflow-great-barrier-reef.zip', 'r') as zip_ref:
#         zip_ref.extractall('tensorflow-great-barrier-reef')
#     os.remove('tensorflow-great-barrier-reef.zip')
    
df = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df['annotations'] = df['annotations'].apply(ast.literal_eval)
df['path'] = df['image_id'].apply(lambda elem : os.path.join('../input/tensorflow-great-barrier-reef/train_images', os.path.join('video_'+str(elem[0]), elem[2:]+'.jpg')))

In [3]:
plt.plot(df['annotations'].apply(len))
plt.ylabel('Number of instance')
plt.xlabel('Frame')
plt.show()

In [4]:
print('Number of frame with one instance :')
print(sum(elem == 1 for elem in df['annotations'].apply(len)))

In [5]:
print('Histogram of number of COTS instances :')
_ = []
for elem in df['annotations'].apply(len):
    if elem >0:
        _.append(elem)
        
plt.hist(_, bins = 18)
plt.ylabel('Number of frame')
plt.xlabel('Number of instance')
plt.show()

In [6]:
TRAIN_RATIO = 0.8

df_test = df.iloc[int(TRAIN_RATIO*df.shape[0]):]
df = df.iloc[:int(TRAIN_RATIO*df.shape[0])]

images_with_cots = df[(df['annotations'].apply(len) > 0)].index
images_without_cots = df[(df['annotations'].apply(len) == 0)].index

MAX_PROPOSALS = 5000
MAX_PROPOSALS_INFER = 5000

MAX_POSITIVE = 30
MAX_NEGATIVE = 3

try:
    os.mkdir('data')
except:
    pass

POSITIVE_PATH = os.path.join(os.getcwd(), os.path.join('data','positive'))
NEGATIVE_PATH = os.path.join(os.getcwd(), os.path.join('data','negative'))
try:
    os.mkdir(POSITIVE_PATH)
    os.mkdir(NEGATIVE_PATH)
except:
    pass

h, w = plt.imread(df.iloc[0]['path']).shape[:2]

INPUT_DIMS = (224, 224)

MODEL_PATH = "cots_detector.h5"
ENCODER_PATH = "label_encoder.pickle"

# define the minimum probability required for a positive prediction
# (used to filter out false-positive predictions)
MIN_PROBA = 0.99

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [7]:
if 'rcnnpreprocess' not in os.listdir('../input'):
    imagePaths = df['path']

    totalPositive = 0
    totalNegative = 0

    for (i, imagePath) in enumerate(imagePaths):
        idx = imagePaths.index[i]
        if len(df.iloc[idx]['annotations']) == 0:
            continue
        print("[INFO] processing image {}/{}...".format(i + 1, len(imagePaths)))

        gtBoxes = [(elem['x'], elem['y'], elem['x']+elem['width'], elem['y']+elem['height']) for elem in df.iloc[i]['annotations']]

        image = cv2.imread(imagePath)
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
        ss.setBaseImage(image)
        ss.switchToSelectiveSearchFast()
        rects = ss.process()
        proposedRects= []
        for (x, y, w, h) in rects:
            proposedRects.append((x, y, x + w, y + h))

        positiveROIs = 0
        negativeROIs = 0
        for proposedRect in proposedRects[:MAX_PROPOSALS]:
            (propStartX, propStartY, propEndX, propEndY) = proposedRect
            for gtBox in gtBoxes:
                iou = compute_iou(gtBox, proposedRect)
                (gtStartX, gtStartY, gtEndX, gtEndY) = gtBox
                roi = None
                outputPath = None

                if iou > 0.7 and positiveROIs <= MAX_POSITIVE:
                    roi = image[propStartY:propEndY, propStartX:propEndX]
                    filename = "{}.png".format(totalPositive)
                    outputPath = os.path.sep.join([POSITIVE_PATH, filename])
                    positiveROIs += 1
                    totalPositive += 1

                fullOverlap = propStartX >= gtStartX
                fullOverlap = fullOverlap and propStartY >= gtStartY
                fullOverlap = fullOverlap and propEndX <= gtEndX
                fullOverlap = fullOverlap and propEndY <= gtEndY

                if not fullOverlap and iou < 0.05 and \
                    negativeROIs <= MAX_NEGATIVE:

                    roi = image[propStartY:propEndY, propStartX:propEndX]
                    filename = "{}.png".format(totalNegative)
                    outputPath = os.path.sep.join([NEGATIVE_PATH, filename])

                    negativeROIs += 1
                    totalNegative += 1

                if roi is not None and outputPath is not None:
                    roi = cv2.resize(roi, INPUT_DIMS,
                        interpolation=cv2.INTER_CUBIC)
                    cv2.imwrite(outputPath, roi)


    # Balance data
    positive_images = os.listdir('positive')
    n_ = len(os.listdir('negative'))
    i = 0
    while len(os.listdir('positive')) < n_:
        elem = random.choice(positive_images)
        shutil.copyfile(os.path.join('positive', elem), os.path.join('positive', str(i)+'_'+elem))
        i += 1
else:
    pass

# Model

In [7]:
INIT_LR = 1e-4
EPOCHS = 50
BS = 8

img_height, img_width = INPUT_DIMS
data_dir = '../input/rcnnpreprocess/data'

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = data_dir, label_mode='categorical',
    class_names=['positive', 'negative'], color_mode='rgb', batch_size=BS, image_size=(224,
    224), shuffle=True, subset="training", seed = 1, validation_split=0.2
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory = data_dir, label_mode='categorical',
    class_names=['positive', 'negative'], color_mode='rgb', batch_size=BS, image_size=(224,
    224), shuffle=True, subset="validation", seed = 1, validation_split=0.2
)

In [9]:
# class_names = train_ds.class_names
# print(class_names)

# AUTOTUNE = tf.data.AUTOTUNE
# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[np.argmax(labels[i])])
#         plt.axis("off")
    
# for image_batch, labels_batch in train_ds:
#     print(image_batch.shape)
#     print(labels_batch.shape)
#     break

In [8]:
model = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom((-0.05, 0.05)),
    layers.RandomTranslation((-0.1, 0.1), (-0.1, 0.1)),
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='sigmoid')
])

print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)

print("[INFO] saving mask detector model...")
model.save(MODEL_PATH, save_format="h5")

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('plot.png')

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model)

# Evaluation

In [ ]:
def predict(img):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()
    
    proposals = []
    boxes = []
    
    for (x, y, w, h) in rects[:MAX_PROPOSALS_INFER]:
        roi = image[y:y + h, x:x + w]
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        roi = cv2.resize(roi, INPUT_DIMS, interpolation=cv2.INTER_CUBIC)
        roi = img_to_array(roi)
        roi = preprocess_input(roi)
        proposals.append(roi)
        boxes.append((x, y, w, h))
        
    proposals = np.array(proposals, dtype="float32")
    boxes = np.array(boxes, dtype="int32")
    
    annot = []
    
    proba = model.predict(proposals)
    for i, elem in enumerate(proba):
        if elem[1] > 0.7:
            annot.append('{:.2f} {} {} {} {}'.format(elem[1], int(boxes[i][0]), int(boxes[i][1]), int(boxes[i][2]), int(boxes[i][3])))
    return ' '.join(annot)



import greatbarrierreef
env = greatbarrierreef.make_env()
iter_test = env.iter_test() 

for (image_np, sample_prediction_df) in iter_test:
    pred = predict(image_np)
    sample_prediction_df['annotations'] = pred
    env.predict(sample_prediction_df)